In [1]:
def mapper() :
    for line in sys.stdin :
        data = line.strip().split("\t")
        date, time, store, item, cost, payment = data
        print ("{0}\t{1}".format(store, cost))

    

데이터가 많을수록 데이터의 예외가 많기 때문에, 어떤 데이터를 받더라도 Mapper가 계속 동작하도록 해야한다.

따라서 MapReudce 코드에서는 Defensive하게 작성하는 것이 중요하다.

In [3]:
# 항목이 6개가 아닐 경우
def mapper() :
    import sys
    
    for line in sys.stdin :
        data = line.strip().split("\t")
        
        if len(data) == 6 :
            date, time, store, item, cost, payment = data
            print ("{0}\t{1}".format(store, cost))


Reducer는 (Key, Value) 쌍을 "Hadoop Streaming"을 통해 받는다. 

Hadoop에서 자체적으로 Shuffle & Sort를 한다. 

In [4]:
def reducer() :
    import sys
    
    salesTotal = 0
    oldKey = None

    for line in sys.stdin :
        data = line.strip().split("\t")
            # store, sales
        if len(data) != 2 :
            continue
            
        thisKey, thisSale = data
            # thisKey = store, thisSale = sale
        
        if oldKey and oldKey != thisKey :
            print ("{0}\t{1}".format(oldKey, salesTotal))
            
            salesTotal = 0
        
        oldKey = thisKey
        salesTotal += float(thisSale)
        
        # 하지만 마지막 Key가 출력되지 않았다 !!
        
        if oldKey != None :
            print ("{0}\t{1}".format(oldKey, salesTotal))

이제 Hadoop을 사용하기 전에, 터미널을 통해 미리 Mapper와 Reducer를 확인해 볼 것이다.

In [21]:
# purchase의 10행만 실험을 위해 사용
! head -n 10 purchases.txt > test.txt
! cat test.txt

2012-01-01	09:00	San Jose	Men's Clothing	214.05	Amex
2012-01-01	09:00	Fort Worth	Women's Clothing	153.57	Visa
2012-01-01	09:00	San Diego	Music	66.08	Cash
2012-01-01	09:00	Pittsburgh	Pet Supplies	493.51	Discover
2012-01-01	09:00	Omaha	Children's Clothing	235.63	MasterCard
2012-01-01	09:00	Stockton	Men's Clothing	247.18	MasterCard
2012-01-01	09:00	Austin	Cameras	379.6	Visa
2012-01-01	09:00	New York	Consumer Electronics	296.8	Cash
2012-01-01	09:00	Corpus Christi	Toys	25.38	Discover
2012-01-01	09:00	Fort Worth	Toys	213.88	Visa


In [22]:
! cat test.txt | python mapper.py

San Jose	214.05
Fort Worth	153.57
San Diego	66.08
Pittsburgh	493.51
Omaha	235.63
Stockton	247.18
Austin	379.6
New York	296.8
Corpus Christi	25.38
Fort Worth	213.88


In [23]:
# 이번에는 Reducer까지 사용
# Sorting은 직접 해준다 (Hadoop에서는 자동)
! cat test.txt | python mapper.py | sort | python reducer.py

Austin	379.6
Corpus Christi	25.38
Fort Worth	367.45
New York	296.8
Omaha	235.63
Pittsburgh	493.51
San Diego	66.08
San Jose	214.05
Stockton	247.18
